In [2]:
import lxw
import pandas as pd
import re

# 1. 客户退货服务

## 1.1 问题背景

你是一名负责友好回应客户询问的客户服务代理。根据以下信息回答客户的问题，不要编造任何关于折扣政策的信息。\
1、退货允许在30天内进行。 // RAG 召回\
2、今天的日期是3月29日。// prompt engineering 或 function 返回\
3、目前所有T恤都有50%的折扣。// RAG 召回 或 function 返回\
4、目前店铺的T恤价格范围是18元到100元。// RAG 召回 或 function 返回

我刚刚在3月1日购买了一件你们 Arnold 系列的T恤。当时它有折扣，所以我买了一件原价30元，打了6折的T恤。我看到你们现在有新的折扣是50%。我想知道是否可以退回那件T恤，并且有足够的退款购买你们的两件T恤。

## 1.2 直接Prompt（IO）

In [11]:
def Direct_IO(question):
    system_prompt = '''
# Profile
你是一名负责友好回应客户询问的客户服务代理。根据 Knowldege 回答客户的问题。

# Knowledge
1. 退货允许在30天内进行。
2. 今天的日期是3月29日。
3. 目前所有T恤都有50%的折扣。
4. 目前店铺的T恤价格范围是18元到100元。

# Constraint
1. 你用中文进行回复。
2. 禁止编造任何关于折扣政策的信息。
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ]

    client = lxw.new_client(model = lxw.gpt4_turbo_1106)

    response = client.chat.completions.create(
        model = lxw.gpt4_turbo_1106,
        messages = messages
    )
    return response.choices[0].message.content

In [16]:
q = '''我刚刚在3月1日购买了一件你们 Arnold 系列的T恤。当时它有折扣，所以我买了一件原价30元，打了6折的T恤。我看到你们现在有新的折扣是50%。我想知道是否可以退回那件T恤，并且有足够的退款购买你们的两件T恤。'''

In [10]:
answer = Direct_IO(q)
print(answer)

尊敬的顾客，很感谢您的咨询。关于您的问题，由于今天是3月29日，您在3月1日购买的Arnold系列T恤购买之日起已超过了我们的30天退货政策的允许期限。因此，您将无法退回那件T恤。

不过，您可以利用我们当前的促销活动，所有T恤都有50%的折扣。根据店铺T恤的价格范围，现在购买T恤的价格会在9元到50元之间。如果您有兴趣购买新的T恤，请您根据您希望购买的款式和价格，应用现有的折扣来计算可能的购买方案。

如果您还有其他问题或需要进一步的帮助，请随时联系我们。谢谢！


## 1.3 Least to Most Prompting

In [12]:
def Decomposition(question):
    system_prompt = '''
# Profile
你是一名负责友好回应客户询问的客户服务代理。根据 Knowldege，分析在回答客户询问之前，必须解决哪些子问题。

# Knowledge
1. 退货允许在30天内进行。
2. 今天的日期是3月29日。
3. 目前所有T恤都有50%的折扣。
4. 目前店铺的T恤价格范围是18元到100元。

# Constraint
1. 你用中文进行回复。
2. 禁止编造任何关于折扣政策的信息。

# Workflow
1. 客户输入要询问的问题
2. 你输出在回答客户询问之前，必须解决哪些子问题的子问题列表。
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ]

    client = lxw.new_client(model = lxw.gpt4_turbo_1106)

    response = client.chat.completions.create(
        model = lxw.gpt4_turbo_1106,
        messages = messages
    )
    return response.choices[0].message.content

In [21]:
def Solve_Subproblems(question, subproblems):
    system_prompt = '''
# Profile
你是一名负责友好回应客户询问的客户服务代理。根据 Knowldege，按照 SUBPROBLEMS 规定的子问题列表，一步一步的解决客户询问。

# Knowledge
1. 退货允许在30天内进行。
2. 今天的日期是3月29日。
3. 目前所有T恤都有50%的折扣。
4. 目前店铺的T恤价格范围是18元到100元。

# Constraint
1. 你用中文进行回复。
2. 你必须一步一步的按照 SUBPROBLEMS 的顺序解决问题。
3. 禁止编造任何关于折扣政策的信息。

# Workflow
1. 用户输入 SUBPROBLEMS 以及要询问的问题
2. 你输出解决过程和最后回复
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": 'SUBPROBLEMS:\r\n' + subproblems + '\r\nINQUIRY:\r\n' + question
        }
    ]

    client = lxw.new_client(model = lxw.gpt4_turbo_1106)

    response = client.chat.completions.create(
        model = lxw.gpt4_turbo_1106,
        messages = messages
    )
    return response.choices[0].message.content

In [27]:
def Solve_Problem(question, analysis_conclusion):
    system_prompt = '''
# Profile
你是一名负责友好回应客户询问的电话客户服务代理。根据资深客服代表（Senior Customer Service Manager）对客户问题的分析结论，回复客户问题。

# Workflow
1. 用户输入客户问题（Customer Inquiry）以及资深客服代表（Senior Customer Service Manager）的分析结论。
2. 你输出最后对客户的回复。要尽量礼貌的保持与客户良好的关系，并与客户产生进一步的互动。

# Constraint
1. 你用中文进行回复。
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": 
                'Customer Inquiry:\r\n' + question + 
                '\r\nSenior Analysis Conclusion of Customer Service Manager:\r\n' + analysis_conclusion
        }
    ]

    client = lxw.new_client(model = lxw.gpt4_turbo_1106)

    response = client.chat.completions.create(
        model = lxw.gpt4_turbo_1106,
        messages = messages
    )
    return response.choices[0].message.content

In [17]:
q = '''我刚刚在3月1日购买了一件你们 Arnold 系列的T恤。当时它有折扣，所以我买了一件原价30元，打了6折的T恤。我看到你们现在有新的折扣是50%。我想知道是否可以退回那件T恤，并且有足够的退款购买你们的两件T恤。'''

In [18]:
decomposed = Decomposition(q)
print(decomposed)

在回答您的问题之前，我们需要解决以下子问题：

1. 顾客购买T恤的日期是否在退货允许的30天内。
2. 当前T恤的实际折扣价格计算，基于店铺目前T恤的价格范围和50%的折扣政策。
3. 根据顾客所提供的原价和折扣信息，计算顾客购买T恤时的实际支付金额。
4. 确定顾客在退货后可用于购买新T恤的预期退款金额。
5. 根据目前的折扣情况，计算顾客能否用退款金额购买两件T恤。


In [22]:
answer = Solve_Subproblems(q, decomposed)
print(answer)

**解决过程:**

1. **顾客购买T恤的日期是否在退货允许的30天内。**
   顾客购买T恤的日期是3月1日，今天是3月29日。3月29日距离3月1日不到30天，因此购买日期仍在退货允许的30天内。

2. **当前T恤的实际折扣价格计算，基于店铺目前T恤的价格范围和50%的折扣政策。**
   店铺T恤的原价范围是18元到100元。根据50%的折扣政策，折扣后的价格范围为：
   - 最低价格: 18元 * 50% = 9元
   - 最高价格: 100元 * 50% = 50元
   因此，现在T恤的实际折扣价格在9元到50元之间。

3. **根据顾客所提供的原价和折扣信息，计算顾客购买T恤时的实际支付金额。**
   顾客提供了原价为30元，并且购买时享受的是6折优惠。
   实际支付金额为：30元 * 60% = 18元。

4. **确定顾客在退货后可用于购买新T恤的预期退款金额。**
   如果顾客退货，按照购买时的实际支付金额进行退款，则退款金额应为18元。

5. **根据目前的折扣情况，计算顾客能否用退款金额购买两件T恤。**
   目前T恤的实际折扣价格最低为9元，因此使用退款金额18元是可以购买两件最低价的T恤的。如果顾客想购买价格更高的T恤，则可能不够支付两件的费用。

**回复:**
您好，您在3月1日购买的T恤可以退货，因为还在30天退货期内。您购买时支付的金额为18元。依照我们现在的折扣活动，T恤的价格在9元到50元之间。因此，您退还原先购买的T恤后得到的18元退款，是可以用来购买两件最低价9元的T恤的。如果您选择价格更高的T恤，退款可能不足以支付两件T恤的费用。希望以上信息对您有帮助！


In [29]:
print(Solve_Problem(q, answer))

亲爱的顾客，

感谢您联系我们！根据您的说明，您于3月1日购买了我们Arnold系列的T恤并享受了6折优惠，因此实际支付了18元。我很高兴通知您，今天是3月29日，您的购买日期仍在我们30天的退货期限内，可执行退货程序。

就您的问题，退还商品后，您的退款金额将是18元。目前店铺的所有T恤享受50%的折扣，价格在9元到50元之间。因此，您将能够用18元的退款购买两件定价在9元的T恤。如果您选择的是更高价位的T恤，那么18元可能不足以覆盖两件T恤的费用。

如果您希望继续进行退货并重新购买，或者有任何其他问题，请告知我们，我们很乐意为您提供更进一步的帮助。

祝购物愉快！

顾客服务团队


# 2. 数据分析任务

In [37]:
def DataDecomposition(question):
    system_prompt = '''
# Profile
你是一名数据分析师。分析在解决数据分析任务之前，必须解决哪些子任务。

# Constraint
1. 你用中文进行回复。
2. 子任务拆解必须到最细颗粒度。

# Workflow
1. 用户输入数据分析任务
2. 你扩展这个任务，是这个任务描述更为精确合理以及结构化
3. 你输出在解决这个任务之前，必须解决哪些子任务的子任务列表。

# Example 1
分析任务：哪个班这次语文考试考的最好？
扩展任务：获取最近一次考试中，语文科目平均成绩最高的班级
子任务列表：
1. 获取所有的班级列表
2. 获取最近一次语文考试的Id
3. 按照最近一次语文考试的Id查询每个班级的平均成绩
4. 对班级按照平均成绩进行排序
5. 输出平均成绩最高的班级
'''
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ]

    client = lxw.new_client(model = lxw.gpt4_turbo_1106)

    response = client.chat.completions.create(
        model = lxw.gpt4_turbo_1106,
        messages = messages
    )
    return response.choices[0].message.content

In [39]:
print(DataDecomposition('C1航道哪个城市的销售额本月同比提升的最多？'))

扩展任务：根据C1航道本月与去年同月的数据，计算哪个城市的销售额同比增长幅度最大

子任务列表：
1. 确定分析涉及的时间范围，即本月及去年同月的具体日期
2. 获取C1航道在确定时间范围内的销售数据
3. 筛选出有关城市的销售数据
4. 计算每个城市在本月的销售总额
5. 计算每个城市在去年同月的销售总额
6. 对每个城市进行销售额同比增长率的计算（本月销售额 - 去年同月销售额）/ 去年同月销售额 * 100%
7. 将城市按同比增长率进行排序
8. 选择出同比提升最多的城市
9. 根据需要进行数据可视化展示，便于理解和报告分析结果
